In [2]:
!pip install --upgrade mxnet-cu102mkl gluoncv
!pip install mxnet-cu100
!pip install Cython
!pip install mmcv

     |████████████████████████████████| 750.3MB 22kB/s 
     |████████████████████████████████| 1.0MB 49.6MB/s 
     |████████████████████████████████| 11.8MB 255kB/s 
     |████████████████████████████████| 317kB 54.8MB/s 
     |████████████████████████████████| 245kB 58.2MB/s 
     |████████████████████████████████| 215kB 56.8MB/s 
     |████████████████████████████████| 102kB 13.3MB/s 
     |████████████████████████████████| 133kB 65.5MB/s 
     |████████████████████████████████| 972kB 47.8MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 675kB 47.4MB/s 
     |████████████████████████████████| 2.6MB 45.1MB/s 
     |████████████████████████████████| 962kB 46.9MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
     |████████████████████████████████| 7.1MB 56.9MB/s 
     |██████████████████████████████

In [3]:
import os
import sys
import argparse
import ast
import cv2
import time
import torch

import matplotlib.pyplot as plt
import numpy as np
import mxnet as mx
from mxnet import gluon, nd, image
from mxnet.gluon.data.vision import transforms
from gluoncv.data.transforms import video
from gluoncv import utils
from gluoncv.model_zoo import get_model

from gluoncv.utils.filesystem import try_import_decord
decord = try_import_decord()

filename = '/content/drive/MyDrive/vin/DeepLearning/final_project/demo/demo.mp4'
device = 'cuda'
output_path = '/content/drive/MyDrive/vin/DeepLearning/final_project/output/'
video_format = 'MJPG'

# define transformations for the video clip
transform_fn = video.VideoGroupValTransform(size=224, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# load pre-trained model
model_name = 'i3d_resnet50_v1_custom'
net = get_model(model_name, nclass=3, pretrained=True)
net.load_parameters(output_path+'/i3d_resnet50_v1_custom_20epochs.params')

# define classes
classes = ['idle', 'walk', 'wave']

if device is not None:
    device = torch.device(device)
else:
    if torch.cuda.is_available():
        torch.backends.cudnn.deterministic = True
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

print('Device:', device)

if filename is not None:
    video = cv2.VideoCapture(filename)
    assert video.isOpened()

    # read video for predicting
    vr = decord.VideoReader(filename)
    
    start_frame = 0
    end_frame = 64
    frame_step = 8
    total_frame = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    video_framerate = int(video.get(cv2.CAP_PROP_FPS))
    print('total_frame:', total_frame)

label_arr = ['str'] * (total_frame+3)
while True:
    if end_frame > total_frame:
        break
    frame_id_list = range(start_frame, end_frame, 2)
    start_frame += frame_step
    end_frame += frame_step
    video_data = vr.get_batch(frame_id_list).asnumpy()
    clip_input = [video_data[vid, :, :, :] for vid, _ in enumerate(frame_id_list)]

    # transform data
    clip_input = transform_fn(clip_input)
    clip_input = np.stack(clip_input, axis=0)
    clip_input = clip_input.reshape((-1,) + (32, 3, 224, 224))
    clip_input = np.transpose(clip_input, (0, 2, 1, 3, 4))

    pred = net(nd.array(clip_input))

    # feed data to the model
    topK = 3
    ind = nd.topk(pred, k=topK)[0].astype('int')
    label = classes[ind[0].asscalar()]
    for i in range(start_frame, end_frame):
        print(i)
        label_arr[i] = label
    # for i in range(topK):
    #     print('\t[%s], with probability %.3f.'%
    #         (classes[ind[i].asscalar()], nd.softmax(pred)[0][ind[i]].asscalar()))
print('label_arr:',label_arr)

video_writer = None
label_cnt = 0
while True:
    t = time.time()

    ret, frame = video.read()
    if not ret:
        break
    label = label_arr[label_cnt]
    label_cnt += 1
    fps = 1. / (time.time() - t)
    print('\rframerate: %f fps' % fps, end='')
    cv2.putText(
                frame, #numpy array on which text is written
                label, #text
                (frame.shape[1] - 150, 50), #position at which writing has to start
                cv2.FONT_HERSHEY_SIMPLEX, #font family
                1, #font size
                (209, 80, 0, 255), #font color
                3) #font stroke
    cv2.imshow('frame.png', frame)
    k = cv2.waitKey(1)
    # if k == 27:  # Esc button
    #     break

    if save_video:
        if video_writer is None:
            fourcc = cv2.VideoWriter_fourcc(*video_format)  # video format
            video_writer = cv2.VideoWriter('output_demo.avi', fourcc, video_framerate, (frame.shape[1], frame.shape[0]))
        video_writer.write(frame)

if save_video:
    video_writer.release()

Streaming output truncated to the last 5000 lines.
429
430
431
432
433
434
435
436
437
438
439
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
4

IndexError: ignored